In [1]:
import tifffile
import numpy as np
import napari

import zarr
import pandas as pd

In [2]:
import dask
from dask.distributed import Client
client = Client() # using local computer
client # view tasks being processed by dask


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 255.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53074,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 255.25 GiB
Comm: tcp://127.0.0.1:53121,Total threads: 6
Dashboard: http://127.0.0.1:53123/status,Memory: 31.91 GiB
Nanny: tcp://127.0.0.1:53077,


In [3]:
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline
from bigstream.piecewise_transform import distributed_apply_transform


In [4]:
spacing = np.array([0.652, 0.652, 0.652])

affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':1000,
    },
}

deform_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(0.25,),
    'control_point_spacing':128.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':100,
    },
}

################## dask ################
blocksize = [64,128,128]
cluster_kwargs = {
    'n_workers':20,
    'threads_per_worker':2,
    'memory_limit':'10GB',
}


In [5]:
fix = tifffile.imread('//nrs.hhmi.org/liu/Hippo_rawtiff/DAPI_s2/padded/stitch_t15_c3_s2_padded.tiff')

In [6]:
################################### make matrix loop ###############################
# we will only use padded data to make transfomration matrix
T = [0,1,2] # for first 3


for t in T:
    print(t)
    mov = tifffile.imread('//nrs.hhmi.org/liu/Hippo_rawtiff/DAPI_s2/padded/stitch_t'+str(t).zfill(2)+'_c3_s2_padded.tiff')
    
    affine = alignment_pipeline(
        fix, mov,
        spacing, spacing,
        steps=[('affine',affine_kwargs)],
    )
    np.savetxt('./_hippo_bigstream_v3/transformation/affine_ref_t15_t'+str(t).zfill(2)+'_c3_s2_padded.mat', affine)
    
    deform = distributed_piecewise_alignment_pipeline(
        fix, mov,
        spacing, spacing,
        steps=[('deform',deform_kwargs)],
        static_transform_list=[affine,],
        blocksize=blocksize,
        write_path='./_hippo_bigstream_v3/transformation/deform_ref_t15_t'+str(t).zfill(2)+'_c3_s2_padded.zarr',
        cluster_kwargs=cluster_kwargs,
    )
    print(t,' : complete')
    

0
Fri Apr 26 18:29:18 2024 Run affine {'alignment_spacing': 2.0, 'shrink_factors': (2,), 'smooth_sigmas': (2.0,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 1000}}
LEVEL:  0  ITERATION:  0  METRIC:  -0.044189049878757186
LEVEL:  0  ITERATION:  1  METRIC:  -0.04466223288286877
LEVEL:  0  ITERATION:  2  METRIC:  -0.045013286777772325
LEVEL:  0  ITERATION:  3  METRIC:  -0.045239240303019505
LEVEL:  0  ITERATION:  4  METRIC:  -0.04551325746267448
LEVEL:  0  ITERATION:  5  METRIC:  -0.045834423228520764
LEVEL:  0  ITERATION:  6  METRIC:  -0.046035412541013225
LEVEL:  0  ITERATION:  7  METRIC:  -0.04631951035504956
LEVEL:  0  ITERATION:  8  METRIC:  -0.04673023888880518
LEVEL:  0  ITERATION:  9  METRIC:  -0.04703655472463089
LEVEL:  0  ITERATION:  10  METRIC:  -0.04721539878867907
LEVEL:  0  ITERATION:  11  METRIC:  -0.04732311444506933
LEVEL:  0  ITERATION:  12  METRIC:  -0.047472500138055915
LEVEL:  0  ITERATION:  13  METRIC:  -0.0477080900767503
LEVEL: 

C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55626 instead
  warnings.warn(


0  : complete
1
Fri Apr 26 19:16:13 2024 Run affine {'alignment_spacing': 2.0, 'shrink_factors': (2,), 'smooth_sigmas': (2.0,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 1000}}
LEVEL:  0  ITERATION:  0  METRIC:  -0.060082414161523816
LEVEL:  0  ITERATION:  1  METRIC:  -0.06100835692261362
LEVEL:  0  ITERATION:  2  METRIC:  -0.061776224139993244
LEVEL:  0  ITERATION:  3  METRIC:  -0.06294855507056653
LEVEL:  0  ITERATION:  4  METRIC:  -0.06415131214353902
LEVEL:  0  ITERATION:  5  METRIC:  -0.06560177356719372
LEVEL:  0  ITERATION:  6  METRIC:  -0.06692185377226675
LEVEL:  0  ITERATION:  7  METRIC:  -0.06814738930575423
LEVEL:  0  ITERATION:  8  METRIC:  -0.06936839347120682
LEVEL:  0  ITERATION:  9  METRIC:  -0.07043173542969945
LEVEL:  0  ITERATION:  10  METRIC:  -0.0713470027614601
LEVEL:  0  ITERATION:  11  METRIC:  -0.07230769252487573
LEVEL:  0  ITERATION:  12  METRIC:  -0.07333657109931019
LEVEL:  0  ITERATION:  13  METRIC:  -0.074424103226463

C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56578 instead
  warnings.warn(


1  : complete
2
Fri Apr 26 19:58:58 2024 Run affine {'alignment_spacing': 2.0, 'shrink_factors': (2,), 'smooth_sigmas': (2.0,), 'optimizer_args': {'learningRate': 0.25, 'minStep': 0.0, 'numberOfIterations': 1000}}
LEVEL:  0  ITERATION:  0  METRIC:  -0.057041410429675476
LEVEL:  0  ITERATION:  1  METRIC:  -0.05783114483031317
LEVEL:  0  ITERATION:  2  METRIC:  -0.05842320050192007
LEVEL:  0  ITERATION:  3  METRIC:  -0.05885506080647364
LEVEL:  0  ITERATION:  4  METRIC:  -0.059434582845117656
LEVEL:  0  ITERATION:  5  METRIC:  -0.05986513856111421
LEVEL:  0  ITERATION:  6  METRIC:  -0.0601274113327513
LEVEL:  0  ITERATION:  7  METRIC:  -0.06049609832492086
LEVEL:  0  ITERATION:  8  METRIC:  -0.06098401295561357
LEVEL:  0  ITERATION:  9  METRIC:  -0.061678946099638005
LEVEL:  0  ITERATION:  10  METRIC:  -0.06241101147309422
LEVEL:  0  ITERATION:  11  METRIC:  -0.06302525287677065
LEVEL:  0  ITERATION:  12  METRIC:  -0.06343122560399066
LEVEL:  0  ITERATION:  13  METRIC:  -0.06360463935247

C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57589 instead
  warnings.warn(


2  : complete


In [8]:
############################ read matrix, transform, crop, save tiff loop #################################
T = [0,1,2] # time points
C = [0,1,2,3] # channels

for t in T:
    # for any given timepoint, the same transformation matrix is used across all channels
    print(t)
    affine_path = './_hippo_bigstream_v3/transformation/affine_ref_t15_t'+str(t).zfill(2)+'_c3_s2_padded.mat'
    affine = np.loadtxt(affine_path)
    deform_path = './_hippo_bigstream_v3/transformation/deform_ref_t15_t'+str(t).zfill(2)+'_c3_s2_padded.zarr'
    deform = zarr.open(deform_path, mode='r')

    # similarly, for any given timepoint, the same N5 path is used, (N5 contains different channels)
    # for mov data, we will use N5 output from bigstitcher
    # padding matters for finding registration matrix, but padding is not necessary for transforming other channels
    setdir = '//nrs.hhmi.org/liu/Jun/output_val_onetile_set1_re1/'
    n5dir = setdir+'/easi/dataset-t'+str(t)+'/stitching/export.n5/'     # path to stitched N5

    print(affine_path)
    print(deform_path)
    print(n5dir)

    # transform one channel at a time using the same transform matrix
    for c in C:
        subpath = 'c'+str(c)+'/s2' # channel loop, including DAPI for checking
        mov_zarr = zarr.open(store=zarr.N5Store(n5dir), mode='r')
        mov = np.array(mov_zarr[subpath]).squeeze() # select channel and resolution, and compute. We have enough memory to hold s2 data

        out_name='./_hippo_bigstream_v3/warped/warped_ref_t15_t'+str(t).zfill(2)+'_c'+str(c)+'_s2_padded'
        print(subpath,'::::',out_name)
        affine_deformed = distributed_apply_transform(
            fix, mov,
            spacing, spacing,
            transform_list=[affine,deform], # affine and then deform
            blocksize=blocksize,
            write_path=out_name+'.zarr',
            cluster_kwargs=cluster_kwargs,
        )
        # we noticed some regions don't have any cells, we will manually crop them out when saving as tiff
        # it's important to keep the cropping elements the same
        tifffile.imwrite(out_name+'_crop.tiff', affine_deformed[30:139, 150:2328, 265:4341], imagej=True, metadata={'axes':'ZYX'})
    print(str(+t),' : complete')
    

0
./_hippo_bigstream_v3/transformation/affine_ref_t15_t00_c3_s2_padded.mat
./_hippo_bigstream_v3/transformation/deform_ref_t15_t00_c3_s2_padded.zarr
//nrs.hhmi.org/liu/Jun/output_val_onetile_set1_re1//easi/dataset-t0/stitching/export.n5/
c0/s2 :::: ./_hippo_bigstream_v3/warped/warped_ref_t15_t00_c0_s2_padded


C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57190 instead
  warnings.warn(


c1/s2 :::: ./_hippo_bigstream_v3/warped/warped_ref_t15_t00_c1_s2_padded


C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57636 instead
  warnings.warn(


c2/s2 :::: ./_hippo_bigstream_v3/warped/warped_ref_t15_t00_c2_s2_padded


...

c2/s2 :::: ./_hippo_bigstream_v3/warped/warped_ref_t15_t02_c2_s2_padded


C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61993 instead
  warnings.warn(


c3/s2 :::: ./_hippo_bigstream_v3/warped/warped_ref_t15_t02_c3_s2_padded


C:\Users\kimj7\AppData\Local\anaconda3\envs\napari_umap_bigstream\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62448 instead
  warnings.warn(


2  : complete
